In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import os.path as path
import random
from IPython.display import HTML

import exdir
import numpy as np
import pandas as pd
from tqdm import tqdm
import skvideo
import yaml

from cow_tus.analysis.visualizations.viewer import play
import cow_tus.data.transforms.preprocessing as transforms

os.chdir('/home/Gangus/cow-tus')

In [3]:
raw_dir = "/data/cow-tus-data/raw"
out_dir = "/data/cow-tus-data/processed"

hypothesis_conditions = ['single-instance-learning', 'temporal-downsample']
group_dir = path.join('data', *hypothesis_conditions)

In [36]:
f = exdir.File('/data/cow-tus-data/processed.exdir')
root_group = f['/'.join(hypothesis_conditions)]

In [94]:
raw_instance_level_df = pd.read_csv(path.join(raw_dir, 'labels_instance_level.csv'))

In [106]:
sum([not np.isnan(dict(raw_instance_level_df.iloc[i])['l5']) for i in range(len(raw_instance_level_df))])

20